#### __Задачи__
Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене).

2. Выявим самый сложный и самый простой экзамен: найдем курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*.

3. По каждому предмету определим средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом).

4. Выявим самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). 

5. Напишем функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выявим семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. 

6. Построим адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. Опишим процесс создания кластеров. Для каждого RFM-сегмента построим границы метрик recency, frequency и monetary для интерпретации этих кластеров.

***завершаемость*** = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

#### __Данные__: 

 __assessments.csv__ — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).
code_module — идентификационный код предмета. 
<br></br>
* __code_presentation__ — семестр (Идентификационный код).

* __id_assessment__ — тест (Идентификационный номер ассессмента).

* __assessment_type__ — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).

* __date__ — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).

* __weight__ — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.
<br></br>
<br></br>
__courses.csv__ — файл содержит список предметов по семестрам.
<br></br>
* __code_module__ — предмет (идентификационный код).
* __code_presentation__ — семестр (идентификационный код).
* __module_presentation_length__ — продолжительность семестра в днях.
<br></br><br></br>
__studentAssessment.csv__ — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.<br></br>

* __id_assessment__ — тест (идентификационный номер).

* __id_studen__ — идентификационный номер студента.

* __date_submitted__ — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.

* __is_banked__ — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).

* __score__ — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.
<br></br><br></br>
__studentRegistration.csv__ — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.
<br></br>
* __code_module__ — предмет (идентификационный код).

* __code_presentation__ — семестр (идентификационный код)

* __id_student__ — идентификационный номер студента.

* __date_registration__ — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

* __date_unregistration__ — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

#### __Импортируем необходимые библиотеки__

In [61]:
import pandas as pd
import numpy as np

import requests
from urllib.parse import urlencode

#### __Импортируем данные и сформируем из них датафреймы__

In [34]:
def get_data(df_url, sep=','):
    """Функция для загрузки данных и их преобразования в датафрейм"""
    
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    
    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=df_url))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его
    df = pd.read_csv(download_url, sep=sep)
    
    return df

In [35]:
assessments = get_data('https://disk.yandex.ru/d/PBW7aUHGuodFDA')
assessments.head(3)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0


In [36]:
courses = get_data('https://disk.yandex.ru/d/m0Z6QYNT46f9tQ')
courses.head(3)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268


In [37]:
student_assessment = get_data('https://disk.yandex.ru/d/lsmdbYB0iM7p3w')
student_assessment.head(3)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0


In [38]:
student_registration = get_data('https://disk.yandex.ru/d/Yse4Y6RJqg_WaA')
student_registration.head(3)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0


#### __1. Сколько студентов успешно сдали только один курс?__

In [39]:
student_assessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [40]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


__Соединим датафреймы assessments и student_assessment, чтобы отфильтровать только успешно сданные экзамены__

In [64]:
temp = pd.merge(assessments, student_assessment)
temp.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,32885,26,0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,38053,19,0,79.0


In [66]:
temp = temp[(temp['assessment_type']=='Exam') & (temp['score'] > 39) & (temp['is_banked'] == 0)]
temp.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0
52928,CCC,2014B,24290,Exam,NaN,100.0,560494,230,0,92.0


In [67]:
temp = (
    temp
    .groupby(['id_student'])
    .agg({'score': 'count'})
    .rename({'score': 'num_of_courses'})
    .score.value_counts()
)
temp

1    3802
2     295
Name: score, dtype: int64

__Ответ: 3706 студентов сдали успешно только 1 курс__

#### __2. Выявим самый сложный и самый простой экзамен: найдем курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*.__
<br></br>
***завершаемость*** = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [68]:
temp = pd.merge(assessments, student_assessment).merge(courses)
temp.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,module_presentation_length
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,268
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70.0,268
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72.0,268
3,AAA,2013J,1752,TMA,19.0,10.0,32885,26,0,69.0,268
4,AAA,2013J,1752,TMA,19.0,10.0,38053,19,0,79.0,268


In [69]:
pass_exam = (
    temp[(temp['score'] > 39.0) & (temp['assessment_type'] == 'Exam')]
    .groupby('id_assessment')
    .agg({'code_module': 'count'})
    .rename(columns={'code_module': 'success_try'})
)
pass_exam

,success_try
id_assessment,
24290,664
24299,1019
25340,504
25354,878
25361,485
25368,842


In [70]:
all_try = (
   temp[temp['assessment_type'] == 'Exam']
    .groupby('id_assessment')
    .agg({'assessment_type': 'count'})
    .rename(columns={'assessment_type': 'total_try'}) 
)
all_try

,total_try
id_assessment,
24290,747
24299,1168
25340,602
25354,968
25361,524
25368,950


In [71]:
pivot = pd.merge(all_try, pass_exam, on='id_assessment')
pivot

,total_try,success_try
id_assessment,,
24290,747,664
24299,1168,1019
25340,602,504
25354,968,878
25361,524,485
25368,950,842


In [72]:
pivot['success_rate'] = round(pivot.success_try / pivot.total_try * 100, 2)

pivot = (
    pivot
    .sort_values('success_rate', ascending=False)
)


In [73]:
pivot.drop(columns=['total_try', 'success_try'], inplace=True)

In [74]:
pivot.idxmin() # самый легкий

success_rate    25340
dtype: int64

In [75]:
pivot.idxmax() # самый сложный

success_rate    25361
dtype: int64

#### __3. По каждому предмету определим средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)__

In [76]:
(
    student_assessment
        .merge(
            assessments[assessments['assessment_type'] == "Exam"][['id_assessment', 'code_module']]
            .drop_duplicates(),
            how='inner',
            on='id_assessment'
            )
        .query("is_banked== 0")
        .groupby('code_module')
        .agg({'date_submitted': np.mean})
)

,date_submitted
code_module,
CCC,239.408877
DDD,237.901445


#### __4. Выявим самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3).__